# 진현영의 텍스트분석 코드

## 전처리

In [1]:
import io
import pandas as pd
import numpy as np

In [11]:
# data load
card_df = pd.read_csv('data/hyun_card.csv', encoding='CP949')

In [22]:
card_df.head(10)

,이용일시,가맹점명,이용금액
0,2020/12/31 20:00,스시안,"19,000"
1,2020/12/30 23:59,티머니 개인택시,"4,500"
2,2020/12/25 19:38,구글플레이,"2,400"
3,2020/12/25 15:03,아르고,"1,000"
4,2020/12/25 12:03,안녕식당,"28,500"
5,2020/12/25 01:06,(주)코리아세븐 천호역점,"19,550"
6,2020/12/24 21:35,지에스 강동천일점,"3,500"
7,2020/12/24 20:43,탐라포차 천호점,"18,400"
8,2020/12/23 13:12,(주)이마트 천호점,"13,860"
9,2020/12/21 11:58,키다리아저씨,"5,300"


In [13]:
# 이용일시, 가맹점명, 이용금액만 남기고 drop
drop_columns = ['승인번호', '본인구분', '브랜드', '이용카드', '이용구분','매입상태']
card_df.drop(drop_columns, axis=1, inplace=True)
card_df.head()

,이용일시,가맹점명,이용금액
0,2020/12/31 20:00,스시안,"19,000"
1,2020/12/30 23:59,티머니 개인택시,"4,500"
2,2020/12/25 19:38,구글플레이,"2,400"
3,2020/12/25 15:03,아르고,"1,000"
4,2020/12/25 12:03,안녕식당,"28,500"


In [14]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [16]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver (1).exe', chrome_options=chrome_options)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [19]:
# https://book.coalastudy.com/data_crawling/week6/stage2
# 네이버 지도 구버전
import time

categories = []

for i in range(10):
    driver.get("https://v4.map.naver.com")
    kword = card_df['가맹점명'][i]
    driver.find_elements_by_css_selector("button.btn_close")[1].click()
    search_box  = driver.find_element_by_css_selector("input#search-input").send_keys(kword)
    search_button = driver.find_element_by_css_selector("button.spm").click()
    stores = driver.find_elements_by_css_selector("div.lsnx")

    for store in stores:
        category = store.find_element_by_css_selector("dd.cate").text
        
    categories.append(category)
    time.sleep(3)

In [20]:
print(categories)

['일식 > 초밥,롤', '일식 > 초밥,롤', '쇼핑,유통 > 상품권판매', '쇼핑,유통 > 카오디오', '술집 > 요리주점', '생활,편의 > 편의점', '생활,편의 > 편의점', '술집 > 포장마차', '쇼핑,유통 > 건강기능보조식품', '부동산 > 중개업']


In [24]:
drop1_index = card_df[card_df['가맹점명'].map(len) == 3].index

card_df['가맹점명'][drop1_index].unique()

array(['가츠시'], dtype=object)

In [25]:
exception_list=['새로이','홈마트','가츠시','그리네','배고팡','팔공티','띵카롱','아비꼬','도네누','카페롱','하얗다','더호떡','꼽사리','연약국','시너리','개미집','위메프']
exception_list

drop_df = pd.DataFrame(card_df['가맹점명'][drop1_index])
drop_df


drop_final_index = drop_df['가맹점명'][~drop_df['가맹점명'].isin(exception_list)].index
card_df = card_df.drop(index=drop_final_index).reset_index().drop('index', axis=1)
card_df

,이용일시,가맹점명,이용금액
0,2020/12/30 23:59,티머니 개인택시,"4,500"
1,2020/12/25 19:38,구글플레이,"2,400"
2,2020/12/25 12:03,안녕식당,"28,500"
3,2020/12/25 01:06,(주)코리아세븐 천호역점,"19,550"
4,2020/12/24 21:35,지에스 강동천일점,"3,500"
...,...,...,...
305,2020/08/04 16:54,세븐일레븐 잠실역사점,"3,900"
306,2020/08/02 18:48,멘야하나비,"23,500"
307,2020/08/02 14:32,롯데물산 (주) 롯데월드몰점,"63,600"
308,2020/08/02 14:30,롯데물산 (주) 롯데월드몰점,"63,600"
